In [1]:
!pip install faiss-cpu sentence-transformers transformers pdfplumber gradio torch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.

In [2]:
# rag_chatbot.py

import pdfplumber
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
import gradio as gr

# 1. Load & chunk the PDF
def load_and_chunk(path, chunk_size=300, overlap=50):
    text = []
    with pdfplumber.open(path) as pdf:
        for page in pdf.pages:
            t = page.extract_text()
            if t:
                text.append(t)
    words = " ".join(text).split()
    chunks = []
    step = chunk_size - overlap
    for i in range(0, len(words), step):
        chunk = words[i : i + chunk_size]
        if chunk:
            chunks.append(" ".join(chunk))
    return chunks

# 2. Embed all chunks once
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# 3. Build in‐memory index (normalized embeddings)
def build_index(chunks):
    embs = embedder.encode(chunks, show_progress_bar=True)
    embs = np.array(embs, dtype="float32")
    norms = np.linalg.norm(embs, axis=1, keepdims=True)
    embs = embs / norms
    return embs

# 4. Load FLAN-T5-Small for generation
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small")
model.eval()

# 5. Retrieval & generation functions
def retrieve(query, chunk_embs, chunks, k=3):
    q_emb = embedder.encode([query], convert_to_numpy=True)
    q_emb = q_emb / np.linalg.norm(q_emb, axis=1, keepdims=True)
    sims = (chunk_embs @ q_emb.T).squeeze()
    topk = np.argsort(-sims)[:k]
    return [chunks[i] for i in topk]

def generate_answer(query, chunk_embs, chunks, k=3, max_new_tokens=128):
    ctxs = retrieve(query, chunk_embs, chunks, k)
    prompt = "Answer the question based on the following contexts:\n\n" + \
             "\n\n".join(f"- {c}" for c in ctxs) + \
             f"\n\nQuestion: {query}\nAnswer:"
    inputs = tokenizer(prompt, return_tensors="pt")
    if torch.cuda.is_available():
        model.to("cuda")
        inputs = {k: v.cuda() for k, v in inputs.items()}
    out = model.generate(**inputs, max_new_tokens=max_new_tokens)
    ans = tokenizer.decode(out[0], skip_special_tokens=True)
    return ans.strip()

# 6. Initialize once
PDF_PATH = r"AI Training Document.pdf"
chunks = load_and_chunk(PDF_PATH)
chunk_embs = build_index(chunks)

# 7. Gradio interface
def answer_fn(query, k):
    return generate_answer(query, chunk_embs, chunks, k=k)

demo = gr.Interface(
    fn=answer_fn,
    inputs=[
        gr.Textbox(label="Your question"),
        gr.Slider(1, 10, value=3, step=1, label="Top-k retrieval")
    ],
    outputs=gr.Textbox(label="Answer"),
    title="RAG Chatbot (FLAN-T5-Small)"
)

if __name__ == "__main__":
    demo.launch()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://da8968cda2b2e95ea5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
